# Async–Await and Fetch API in JavaScript 

## Introduction

This lecture explains how `async/await` helps write cleaner asynchronous code in JavaScript and how to use the Fetch API to make HTTP requests like GET and POST.  It uses practical examples with fake APIs such as JSONPlaceholder to show how data is fetched, parsed, and processed.[1][2][3][4]

***

## Promises recap

- A Promise is an object representing the eventual completion or failure of an asynchronous operation.[5][6]
- It has two main final states:
  - Fulfilled (resolved successfully).
  - Rejected (failed).[5]
- Typical handling:
  - `promise.then(...)` for success.
  - `promise.catch(...)` for errors.
  - `promise.finally(...)` for cleanup in both cases.[5]

Problems when using many `then` calls:
- Long chains of `then` can become hard to read.  
- Debugging deeply chained or nested promises is difficult.[7]

This motivates using `async/await` for cleaner, more sequential-looking code.[1][7]

***

## Synchronous vs Asynchronous code

### Synchronous code

- Code runs line by line in order.  
- Example idea:  
  - Line 1: `console.log("A")`  
  - Line 2: `console.log("B")`  
  - Line 3: `console.log("C")`  
- Execution: A runs completely, then B, then C. The next line never starts until the current one is done.[8]

### Asynchronous code

- Some operations (like network calls, timers) take time and do not block the main thread.[2][8]
- Example pattern:
  - Line 1: Start async operation (e.g., HTTP request or `setTimeout`).  
  - JS continues to next lines immediately while the async task is still running.  
- The async operation completes later and its callback or promise handler runs when the result is ready.[8][2]

***

## What async–await does conceptually

### Purpose

- `async/await` allows asynchronous code to look and behave more like synchronous, line-by-line code.[9][1]
- It improves:
  - Readability (code looks sequential).
  - Debuggability.
  - Clean handling of multiple asynchronous steps.[7]

### Key rules

- `async` is used before a function definition:
  - `async function myFunc() { ... }`[6]
- An `async` function **always returns a Promise**:
  - Returning a value → Promise resolved with that value.
  - Throwing an error → Promise rejected with that error.[10][6]
- `await`:
  - Can be used **only inside an async function**.[6][1]
  - `let value = await somePromise;`  
  - Pauses the async function at that line until the promise is fulfilled or rejected, then resumes with the result or throws the error.[9][1]

### Intuition (relay race analogy)

- Think of multiple async steps as runners in a relay race.
- Each `await` waits until its “runner” finishes and passes the baton (the result) before moving to the next step.[9]
- So:
  - First async step finishes.
  - Then the second starts.
  - Then the third, and so on.  
- This is like enforcing sequential order on naturally asynchronous operations.

***

## Creating an async function (basic example)

Conceptual example:

- Define an async function:

  - Use `async function getData() { ... }`.[6]
  - Inside, you can still use normal synchronous code (like `console.log`) and asynchronous code (like `setTimeout`, Fetch calls, etc.).

- `setTimeout` example:
  - Inside `getData`, a `setTimeout` simulates an async task that runs after 3 seconds and logs a message.  
  - When `getData()` is called, the function starts, schedules the timeout, and returns a Promise that will be settled when the async work is done.[8]

- If you store the return value of an async function in a variable and log it directly, you see a Promise object, confirming that async functions return Promises.[10][6]

***

## When and why to use await

### The core idea

`await` is used when you want the next line of code to run **only after** an asynchronous operation completes.[1][9]

Example pattern:

- Step 1: Build a URL or endpoint string (generally synchronous).  
- Step 2: Call Fetch (asynchronous network call).[2]
- Step 3: Process the data (often synchronous, but may involve async parsing such as `response.json()`).[2]

Problem without `await`:
- If you call Fetch and then immediately try to use the data, the data may not have arrived yet.  
- This often leads to `undefined` or incomplete data being processed.[2]

Using `await`:
- `let response = await fetch(url);`  
  - Execution pauses at this line until the HTTP request completes and a response is available.[1][2]
- `let data = await response.json();`  
  - Execution pauses again until the body is fully parsed into a JavaScript object/array.[2]

Thus, the code behaves more like:

1. Wait until response is ready.  
2. Then wait until parsing is done.  
3. Then use the final data.  

This provides a “synchronous-style” flow on top of asynchronous operations.[9][1]

***

## Fetch API basics

### What is Fetch?

- The Fetch API is a modern interface for making HTTP requests (GET, POST, PUT, DELETE, etc.) from JavaScript.[11][2]
- It replaces older patterns like `XMLHttpRequest` with a cleaner promise-based interface.[11]

### Available HTTP methods

Common request methods used with Fetch:[12][2]

- GET – Retrieve data (read).  
- POST – Create new data.  
- PUT/PATCH – Update existing data.  
- DELETE – Remove data.

### Basic GET request syntax

General pattern:

```js
const response = await fetch(url);
const data = await response.json();
```

- `fetch(url)` returns a Promise that resolves to a `Response` object.[2]
- `response.json()` returns another Promise that resolves to the parsed JSON data.[2]

***

## Using JSONPlaceholder and similar fake APIs

### What is JSONPlaceholder?

- JSONPlaceholder is a free fake REST API used for testing and learning HTTP requests.[3][4]
- It exposes endpoints like:
  - `/posts`
  - `/comments`
  - `/todos`
  - `/users`, etc.[13][3]

Example endpoints:[4][3]

- `https://jsonplaceholder.typicode.com/todos`  
- `https://jsonplaceholder.typicode.com/posts`  
- `https://jsonplaceholder.typicode.com/comments`

These endpoints return JSON data such as arrays of objects (for lists) or single objects (for specific resources).[13][4]

***

## GET requests with Fetch + async–await

### Example: basic GET

Conceptual steps inside an async `getData` function:

1. Prepare the URL  
   - Example: `const url = "https://jsonplaceholder.typicode.com/posts";`[3][13]

2. Fetch the data
   - `const response = await fetch(url);`  
   - This is an asynchronous network call, so `await` is used to wait for the response.[2]

3. Parse the JSON
   - `const data = await response.json();`  
   - Parsing is also asynchronous and returns a Promise, so `await` ensures the full data is ready.[2]

4. Use the data
   - For example: `console.log(data);`  
   - For `/posts`, this is an array of post objects (each with id, userId, title, body).[4][13]
   - For `/comments`, this is an array of comment objects.

Key points:

- Without `await`, code would try to log/process `data` before the Promise is resolved.  
- Adding `await` gives a clean top‑down flow: first fetch completes, then parsing, then processing.[1][9]

***

## Understanding JSON responses

- Many APIs (including JSONPlaceholder) return data in JSON (JavaScript Object Notation) format.[11][4]
- Typical shapes:
  - Array of objects (e.g., list of posts or comments).  
  - Single object for a specific ID.  
- Example structure for `/posts` from JSONPlaceholder:  
  - `[{ id: 1, userId: 1, title: "...", body: "..." }, ...]` (conceptually; actual text not reproduced).[13][4]

When logged to the console, you can see:

- The overall array length.  
- Each object’s fields like `userId`, `id`, `title`, `body`.[13]

***

## Rules and constraints for using await

- `await` can only be used inside an `async` function.[6][1]
- If you try to use `await` at the top level in environments that don’t support top‑level await, or inside a non‑async function, you get a syntax error.[6][1]
- Any async operation you want to behave sequentially should be `await`ed:
  - HTTP requests (`fetch`).
  - Parsing (`response.json()` if it returns a Promise).[2]

This is why both steps:

- `const response = await fetch(url);`  
- `const data = await response.json();`

are typically awaited.

***

## POST requests with Fetch

### Why POST needs extra options

- A GET request often needs only the URL.  
- POST/PUT/PATCH/DELETE usually require additional information, such as:
  - Request method (`method: "POST"`).
  - Request body (the data to send).
  - Headers (like `Content-Type: "application/json"`).[12][2]

### Basic POST pattern

General structure:

```js
const url = "https://...";  // API endpoint
const options = {
  method: "POST",
  headers: {
    "Content-Type": "application/json"
  },
  body: JSON.stringify({
    // data to send
  })
};

const response = await fetch(url, options);
const data = await response.json();
```

- `options` is an object describing method, headers, body, and possibly other properties (credentials, mode, etc.).[12][2]
- The body must be serialized as a JSON string when sending JSON data.[12][2]

### Example with fake APIs

- Some training APIs (including JSONPlaceholder variants and dummy JSON services) accept POST requests but do not permanently store the new data; they respond with a “fake” created resource that is not actually persisted.[3][4]
- You may see:
  - The POST response showing the data you sent plus a new `id`.  
  - A later GET by that `id` might not show the new record because the service does not truly maintain state.[4][3]

Important implication:
- POST success is demonstrated by the response to the POST request itself, not by expecting long‑term persistence on some fake APIs.[3][4]

***

## Building options objects for different methods

- For GET with options, you might specify headers or other configurations but often omit `body`.[2]
- For POST:
  - `method: "POST"`.
  - `headers: { "Content-Type": "application/json" }`.
  - `body: JSON.stringify(dataObject)`.[12][2]
- For PUT/DELETE:
  - Use `method: "PUT"` or `method: "DELETE"` and follow similar patterns.[12][2]

Documentation (MDN and others) is typically used as a reference rather than memorizing all possible properties.[6][2]

***

## Combining multiple async operations with async–await

### Sequential operations

A typical pattern:

```js
async function processData() {
  const postResult = await postData();     // 1) send POST
  const allData = await getData();         // 2) get updated data
  // 3) Process both results
}
```

- `postData()` and `getData()` themselves are async functions wrapping Fetch calls.  
- `await` ensures:
  - First, the POST completes.  
  - Then, the GET runs.  
- This composes multiple async steps in a clear, readable way.[9][1]

### Relay race analogy revisited

- Each `await` line is like waiting for the current “runner” to finish before starting the next.[9]
- If `A`, `B`, and `C` are async operations:

  - Wait for `A` to finish (`await A`).
  - Then wait for `B` (`await B`).
  - Then wait for `C` (`await C`).  

- This mimics normal synchronous control flow while still using non‑blocking async operations underneath.[1][9]

***

## Error handling with async–await (conceptual)

While not heavily emphasized in the transcript, the standard pattern is:

- Use `try...catch` inside async functions:
  - `try` contains `await` calls.  
  - `catch` handles any rejection thrown as an exception.[14][7]

This replaces `.catch()` chains on Promises with more familiar `try...catch` blocks.

***

## Key concepts table

| Concept                | What it means                                                                                         | Important points                                                                                         |
|------------------------|------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| Promise                | Object representing eventual completion/failure of async operation                                   | States: pending → fulfilled or rejected; handled via `then`, `catch`, `finally`. [5]                |
| `async` function       | Function declared with `async` keyword that returns a Promise                                        | Allows use of `await` inside; always returns a Promise (resolved or rejected). [10][6]        |
| `await`                | Keyword that pauses an async function until a Promise settles                                        | Only valid inside async functions; returns the resolved value or throws on rejection. [1][9]   |
| Fetch API              | Web API for making HTTP requests                                                                     | Uses Promises; `fetch(url, options)` returns a `Response` object. [11][2]                       |
| GET request            | Request to fetch/read data                                                                           | Usually just needs URL; often followed by `response.json()` to parse. [2][12]                  |
| POST request           | Request to create new data                                                                           | Needs method, headers, and body (often JSON). [2][12]                                          |
| JSONPlaceholder / mock APIs | Fake REST APIs that provide sample data and often non‑persistent writes                        | Great for learning GET/POST/PUT/DELETE patterns. [3][4]                                       |
| `response.json()`      | Method on Fetch Response to parse JSON body                                                          | Returns a Promise that resolves to JS objects/arrays. [2]                                          |

***

## Practical study tips for this topic

- Always wrap `await` calls inside `async` functions; if an error appears, first check if the function is marked as `async`.[1][6]
- For debugging:
  - Log intermediate values: responses, parsed data, URLs, and options objects.  
  - Check network tab in dev tools to see request URLs, methods, payloads, and responses.[2]
- Practice by calling multiple endpoints (posts, comments, todos) and writing helper functions:
  - `getPosts()`, `getComments()`, `createPost()`, etc., all using Fetch with async–await.[3][2]

***

## Summary of main takeaways

- `async/await` is special syntax that makes working with Promises easier and code more readable by giving async code a synchronous look.[7][1]
- An `async` function always returns a Promise, and `await` pauses that function until a Promise settles, enabling clean step‑by‑step flows for tasks like network requests and JSON parsing.[10][6]
- The Fetch API is a modern way to make HTTP requests; common patterns combine Fetch with async–await for GET and POST operations, often against mock APIs like JSONPlaceholder for practice.